<!--NOTEBOOK_HEADER-->
*This notebook contains course material from [FAT0413156](https://gmxavier.github.io/FAT0413156)
by Gilberto Xavier (gilberto.xavier at lps.ufrj.br); the content is available [on Github](https://github.com/gmxavier/FAT0413156.git).
The text is released under the [CC-BY-NC-SA-4.0 license](https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode),
and code is released under the [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Flow measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/06.00-Flow.ipynb) | [Contents](toc.ipynb) | [Analytical measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/08.00-Analytical.ipynb) ><p><a href="https://colab.research.google.com/github/gmxavier/FAT0413156/blob/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/gmxavier/FAT0413156/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# Control and safety valves

## Control valves

Reading: Tony R. Kuphaldt's [*Lessons in Industrial Instrumentation*: Chapter 27](https://www.ibiblio.org/kuphaldt/socratic/sinst/book/liii_2v32.pdf#page=2110)

Instructions: Click the link above and read this chapter. Chapter 27 describes the main control valve types used as final control elements.

Reading this chapter should take approximately ?? minutes.

Terms of use: This work is licensed under terms of the [CC-BY-4.0 license](http://creativecommons.org/licenses/by/4.0/). It is attributed to Tony R. Kuphaldt.

Supplementary material:
- [Control valve at Wikipedia](https://en.wikipedia.org/wiki/Control_valve)
- [Emerson's Control Valve Handbook](https://www.emerson.com/documents/automation/control-valve-handbook-en-3661206.pdf)

## Safety valves

Reading: Tony R. Kuphaldt's [*Lessons in Industrial Instrumentation*: Section 32.5](https://www.ibiblio.org/kuphaldt/socratic/sinst/book/liii_2v32.pdf#page=2687)

Instructions: Click the link above and read this section. Section 32.5 provides a glance about overpressure safety devices including safety valves.

Reading this section should take approximately ?? minutes.

Terms of use: This work is licensed under terms of the [CC-BY-4.0 license](http://creativecommons.org/licenses/by/4.0/). It is attributed to Tony R. Kuphaldt.

## Exercises

Try to solve the related problems listed [here](https://doc.lagout.org/electronics/Fundamentals%20of%20Industrial%20Instrumentation%20and%20Process%20Control%20%5Bby%20William%20Dunn%5D.pdf) on page 198.

<!--NAVIGATION-->
< [Flow measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/06.00-Flow.ipynb) | [Contents](toc.ipynb) | [Analytical measurement](http://nbviewer.jupyter.org/github/gmxavier/FAT0413156/blob/master/notebooks/08.00-Analytical.ipynb) ><p><a href="https://colab.research.google.com/github/gmxavier/FAT0413156/blob/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://raw.githubusercontent.com/gmxavier/FAT0413156/master/notebooks/05.00-Temperature.ipynb" target="_parent"><img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>